# Understanding Hired Rides in NYC
## IEOR E4501 Final Project

## Project Setup

In [ ]:
# all import statements needed for the project
import math
import os

import bs4
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.axes import Axes
from matplotlib.container import BarContainer

import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.distance import distance
import requests
import re
import sqlalchemy as db

import folium
from folium.plugins import HeatMap

from typing import List, Union, Any, Dict,  Tuple
from matplotlib.container import BarContainer

In [ ]:
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
TAXI_DIR = "data/taxi"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_CSV = "data/uber_rides_sample.csv"
WEATHER_CSV_DIR = "data/weather"

CRS = 4326 # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
    print(f"Folder {QUERY_DIRECTORY} created successfully.")
except FileExistsError:
    print(f"Folder {QUERY_DIRECTORY} already exists.")

In [ ]:
# Make sure the TAXI_DIR exists
try:
    os.mkdir(TAXI_DIR)
    print(f"Folder {TAXI_DIR} created successfully.")
except FileExistsError:
    print(f"Folder {TAXI_DIR} already exists.")

## Part 1: Data Preprocessing

### Load Taxi Zones

In [ ]:
def load_taxi_zones(shapefile: str) -> pd.DataFrame:
    """Load the shapefile and return it as a DataFrame.
    
    Parameters
    ----------
    shapefile : str
        The relative path of the shape file including zone IDs and geometries.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe including the geometries and corresponding zone ID.
    
    """
    loaded_taxi_zones = gpd.read_file(shapefile)
    loaded_taxi_zones = loaded_taxi_zones[['OBJECTID', 'geometry']].set_index('OBJECTID')
    # Transform geometries to the new coordinate reference system 4326
    loaded_taxi_zones = loaded_taxi_zones.to_crs(CRS)
    
    return loaded_taxi_zones

In [ ]:
def lookup_coords_for_taxi_zone_id(zone_loc_id: int,
                                   loaded_taxi_zones: pd.DataFrame) -> tuple:
    """Given the zone ID and return the corresponding centroid coordinates.
    
    Parameters
    ----------
    zone_loc_id : int
        The zone ID which needs to be searched.
    loaded_taxi_zones : pandas.DataFrame
        A dataframe including the geometries and corresponding zone ID.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe including the geometries and corresponding zone ID.

    """
    geometry = loaded_taxi_zones.loc[zone_loc_id, 'geometry']
    # Obtain the approximate coordinates by the centroid location
    longitude = geometry.centroid.x
    latitude = geometry.centroid.y
    coords = (longitude, latitude)
    
    return coords

### Calculate distance

In [ ]:
def calculate_distance_with_coords(from_coord: tuple, to_coord: tuple) -> float:
    """Given the coordinates and return the distance between them.
    
    This function utilizes the Haversine formula to calculate the distance
    between two coordinates on Earth's surface.
    
    Parameters
    ----------
    from_coord : tuple of float
        A tuple containing the longitude and latitude of the starting point, expressed in degrees.
    to_coord : tuple of float
        A tuple containing the longitude and latitude of the destination point, expressed in degrees.
    
    Returns
    -------
    float
        The distance between the two coordinates, in kilometers.
        
    """
    
    # Convert the input coordinates from degrees to radians
    from_lon, from_lat = math.radians(from_coord[0]), math.radians(from_coord[1])
    to_lon, to_lat = math.radians(to_coord[0]), math.radians(to_coord[1])
    # Calculate the differences in latitude and longitude
    delta_lon = to_lon - from_lon
    delta_lat = to_lat - from_lat
    # Apply the Haversine formula to calculate the distance
    a = math.sin(delta_lat / 2)**2 + math.cos(from_lat) * math.cos(to_lat) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = 6371 * c # earth's radius is assumed to be 6371 kilometers
    
    return distance

In [ ]:
def add_distance_column(dataframe: pd.DataFrame) -> pd.DataFrame:
    """Add the 'distance' column to the dataframe.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The dataframe which needs to be processed.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe including the new calculated column 'distance'.
    
    """
    dataframe['distance'] = dataframe.apply(lambda x:calculate_distance_with_coords(
                                (x['pickup_longitude'],x['pickup_latitude']),
                                (x['dropoff_longitude'],x['dropoff_latitude'])), axis=1)
    
    return dataframe

### Remove outside trips

In [ ]:
def remove_outside_trip(df: pd.DataFrame) -> pd.DataFrame:
    """Remove the trip records outside the defined region.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe which needs to be processed.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after removing all trip records outside the defined region.
    
    """
    # Obtain the coordinate limits
    southlimit, westlimit = NEW_YORK_BOX_COORDS[0]
    northlimit, eastlimit = NEW_YORK_BOX_COORDS[1]
    # Remove the trips outside the location 
    df = df[(df['pickup_longitude'] >= westlimit) & (df['pickup_longitude'] <= eastlimit)]
    df = df[(df['pickup_latitude'] >= southlimit) & (df['pickup_latitude'] <= northlimit)]

    df = df[(df['dropoff_longitude'] >= westlimit) & (df['dropoff_longitude'] <= eastlimit)]
    df = df[(df['dropoff_latitude'] >= southlimit) & (df['dropoff_latitude'] <= northlimit)]
    
    return df

### Processing Taxi Data

In [ ]:
def get_all_urls_from_taxi_page(taxi_page: str) -> list:
    """Scrap the URLs from the given page and return them as a list.
    
    Parameters
    ----------
    taxi_page : str
        The URL of the target page.
    
    Returns
    -------
    list
        A list of URLs scraped from the given page.
        
    """
    all_urls = list()
    
    content = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(content.text, 'lxml')
    # Find all the URLs in the page
    for link in soup.find_all("a"):
        all_urls.append(link.get('href'))
        
    return all_urls

In [ ]:
def filter_taxi_parquet_urls(all_urls: list) -> list:
    """Find the URLs for yellow taxi data and return them as a list.
    
    Parameters
    ----------
    all_urls : str
        A list of URLs.
    
    Returns
    -------
    list
        A list of URLs for yellow taxi parquet files from 2019-01 to 2015-06.
        
    """
    all_parquet_urls = list()
    
    pattern = r".*yellow_tripdata.*parquet\Z"
    time_pattern = r"(2009-(0[1-9]|1[0-2]))|(201[0-4]-(0[1-9]|1[0-2]))|(2015-(0[1-6]))"
    for url in all_urls:
        # Check if the URL belongs to yellow taxi trip data
        if re.search(pattern, url):
            # Check if the URL belongs to the time range for the project
            if re.search(time_pattern, url):
                all_parquet_urls.append(url)
            
    return all_parquet_urls

In [ ]:
def generate_coords_from_zones(dataframe):
    """Generate the coordinates from zone IDs in a DataFrame.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe which needs to be processed.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after transforming zone IDs to longitude and latitude coordinates.
    
    """
    loaded_taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    southlimit, westlimit = NEW_YORK_BOX_COORDS[0]
    northlimit, eastlimit = NEW_YORK_BOX_COORDS[1]
    
    for index, row in dataframe.iterrows():
        pickup_zoneid = row['pickup_zoneid']
        dropoff_zoneid = row['dropoff_zoneid']

        pickup_coords = lookup_coords_for_taxi_zone_id(pickup_zoneid, loaded_taxi_zones)
        
        # define the initial bearing
        direction = 0
        # check if pickup and dropoff zones are the same
        if pickup_zoneid == dropoff_zoneid:
            # generate dropoff coordinates using distance and bearing
            dropoff_coords = distance(
                miles=row['trip_distance']).destination(pickup_coords[::-1], bearing=direction)[1::-1]
        else:
            # generate dropoff coordinates using dropoff zone ID
            dropoff_coords = lookup_coords_for_taxi_zone_id(dropoff_zoneid, loaded_taxi_zones)
        
        # check if dropoff coordinates fall outside the defined box
        while not ((westlimit <= dropoff_coords[0] <= eastlimit) and 
                   (southlimit <= dropoff_coords[1] <= northlimit)):
            # Generate new dropoff coordinates by changing the bearing
            direction += 90
            # If all four bearings do not work, drop this record instead
            if direction == 360:
                break
            dropoff_coords = distance(
                miles=row['trip_distance']).destination(pickup_coords, bearing=direction)[1::-1]
        
        # update the dataframe with the generated coordinates
        if direction != 360:
            dataframe.loc[index, 'pickup_longitude'] = pickup_coords[0]
            dataframe.loc[index, 'pickup_latitude'] = pickup_coords[1]
            dataframe.loc[index, 'dropoff_longitude'] = dropoff_coords[0]
            dataframe.loc[index, 'dropoff_latitude'] = dropoff_coords[1]
        else:
            dataframe.drop(index=index, inplace=True)
    
    # Drop the unnecessary columns
    dataframe.drop(['trip_distance', 'pickup_zoneid', 'dropoff_zoneid'], axis=1, inplace=True)
    
    return dataframe

In [ ]:
def clean_taxi_df_2009_to_2010(dataframe: pd.DataFrame) -> pd.DataFrame:
    """Clean the yellow taxi data from 2009 to 2010.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe with records from 2009 to 2010.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after normalizing column names, removing invalid data, and sampling.
        
    """
    # Normalize the column names
    dataframe.columns = ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
                         'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',
                         'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 
                         'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 
                         'tolls_amount', 'total_amount']
    
    # Remove the trips outside the required coordinate box
    dataframe = remove_outside_trip(dataframe)
    # Remove the trips with zero passenger count
    dataframe = dataframe[dataframe['passenger_count'] != 0]
    # Remove the trips without a fare
    dataframe = dataframe[dataframe['fare_amount'] != 0]
    # Remove the trips with no distance between pickup and dropoff locations
    dataframe = dataframe[dataframe['trip_distance'] != 0]
    dataframe = dataframe[(dataframe['pickup_longitude'] != dataframe['dropoff_longitude']) 
                          & (dataframe['pickup_latitude'] != dataframe['dropoff_latitude'])]
    
    # Sample the taxi data at a appropriate number
    dataframe = dataframe.sample(n = 2500, random_state=1)
    
    # Choose useful columns for the coming analysis
    columns_to_keep = ['pickup_datetime', 'pickup_longitude', 'pickup_latitude',
                       'dropoff_longitude', 'dropoff_latitude', 'tip_amount']
    dataframe = dataframe[columns_to_keep]
    
    # Transform the pickup datetime column from strings to datetime
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'],
                                                  format='%Y-%m-%d %H:%M:%S') 
    
    return dataframe

In [ ]:
def clean_taxi_df_2011_to_2015(dataframe: pd.DataFrame) -> pd.DataFrame:
    """Clean the yellow taxi data from 2011 to 2015.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe with records from 2011 to 2015.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after normalizing column names, removing invalid data, and sampling.
        
    """
    # Normalize the column names
    dataframe.columns = ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
                         'trip_distance', 'rate_code', 'store_and_fwd_flag', 'pickup_zoneid',
                         'dropoff_zoneid', 'payment_type', 'fare_amount', 'surcharge', 'mta_tax',
                         'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
                         'congestion_surcharge', 'airport_fee']
    
    # Remove the trips outside the 1-263 zones
    dataframe = dataframe[dataframe['pickup_zoneid'] != 264]
    dataframe = dataframe[dataframe['pickup_zoneid'] != 265]
    dataframe = dataframe[dataframe['dropoff_zoneid'] != 264]
    dataframe = dataframe[dataframe['dropoff_zoneid'] != 265]
    # Remove the trips with zero passenger count
    dataframe = dataframe[dataframe['passenger_count'] != 0]
    # Remove the trips without a fare
    dataframe = dataframe[dataframe['fare_amount'] != 0]
    # Remove the trips with no distance between pickup and dropoff locations
    dataframe = dataframe[dataframe['trip_distance'] != 0]
    
    # Sample the taxi data at a appropriate number
    dataframe = dataframe.sample(n = 2500, random_state=1)
    
    # Choose useful columns for the coming analysis
    columns_to_keep = ['pickup_datetime', 'trip_distance',
                       'pickup_zoneid', 'dropoff_zoneid', 'tip_amount']
    dataframe = dataframe[columns_to_keep]
    
    # Generate the coordinates from zone IDs
    dataframe = generate_coords_from_zones(dataframe)
    
    return dataframe

In [ ]:
def get_and_clean_month(url: str) -> pd.DataFrame:
    """Load and clean the parquet file for the URL, return it as a DataFrame.
    
    Parameters
    ----------
    url : str
        The URL for the parquet file.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after loading and cleaning the parquet file from the given URL.
        
    """
    # Programmatically download needed data if not exists
    dataframe = pd.DataFrame()
    
    time_pattern = r"(2009-(0[1-9]|1[0-2]))|(201[0-4]-(0[1-9]|1[0-2]))|(2015-(0[1-6]))"
    time = ""
    
    if re.search(time_pattern, url):
        time = re.search(time_pattern, url).group(0)
        file_path = f"{TAXI_DIR}/yellow_taxi_{time}.parquet"
        
        # Check if the parquet file has already been downloaded
        if os.path.exists(file_path):
            print(f"File {file_path} already exists.")
            dataframe = pd.read_parquet(file_path, engine='pyarrow')
        else:
            # If not, download the file from the given URL
            print(f"File {file_path} does not exist. Downloading...")
            response = requests.get(url, stream=True)
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
                print(f"File {file_path} downloaded successfully.")
            dataframe = pd.read_parquet(file_path, engine='pyarrow')
    
    # Clean the dataframe
    if re.search(r"2009|2010", time):
        dataframe_cleaned = clean_taxi_df_2009_to_2010(dataframe)
    else:
        dataframe_cleaned = clean_taxi_df_2011_to_2015(dataframe)
    
    return dataframe_cleaned

In [ ]:
def get_and_clean_taxi_data(parquet_urls: list) -> pd.DataFrame:
    """Preprocess and concatenate all the data, return them as a DataFrame.
    
    Parameters
    ----------
    parquet_urls : list
        A list of URLs for parquet files of yellow taxi data.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after preprocessing and concatenating all the parquet file data.
    
    """
    all_taxi_dataframes = []
    
    # Iterate the URLs and obtain the dataframe for each month
    for parquet_url in parquet_urls:
        dataframe = get_and_clean_month(parquet_url)
        # Add the 'distance' column
        dataframe = add_distance_column(dataframe)
        all_taxi_dataframes.append(dataframe)

    # Create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    return taxi_data

In [ ]:
def get_taxi_data() -> pd.DataFrame:
    """Scrap the yellow taxi data and return the result as a DataFrame.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe including all cleaned and sampled records for yellow taxi data.
    
    """
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = filter_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

In [ ]:
# Download the sampled yellow taxi data as a CSV file
taxi_data.to_csv('data/taxi/yellow_taxi_sampled.csv', index=False)
# Load the sampled yellow taxi data directly
taxi_data = pd.read_csv('data/taxi/yellow_taxi_sampled.csv')

### Processing Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file: str) -> pd.DataFrame:
    """Load and clean the Uber data and return it as a DataFrame.
    
    Parameters
    ----------
    csv_file : str
        The relative path of the CSV file of Uber data.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after loading and cleaning all the Uber data.
    
    """
    columns_to_keep = ['pickup_datetime', 'pickup_longitude', 'pickup_latitude',
                       'dropoff_longitude', 'dropoff_latitude']
    dataframe = pd.read_csv(csv_file, usecols = columns_to_keep)
    # Transform the pickup datetime column from strings to datetime
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'],
                                                  format='%Y-%m-%d %H:%M:%S %Z')
    dataframe['pickup_datetime'] = dataframe['pickup_datetime'].dt.tz_convert(None)
    
    # Remove the trips outside the defined coordinate box
    dataframe = remove_outside_trip(dataframe)
    # Remove the trips with no distance between pickup and dropoff locations
    dataframe = dataframe[(dataframe['pickup_longitude'] != dataframe['dropoff_longitude']) 
                          & (dataframe['pickup_latitude'] != dataframe['dropoff_latitude'])]
    
    return dataframe

In [ ]:
def get_uber_data() -> pd.DataFrame:
    """Return the processed Uber data as a DataFrame.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe after preprocessing all the Uber data and adding column 'distance'.
    
    """
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    # Add the 'distance' column
    add_distance_column(uber_dataframe)
    uber_dataframe.dropna(axis=0, inplace=True)
    
    return uber_dataframe

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

### Processing Weather Data

In [ ]:
def drop_invalid_rows_for_HourlyWindSpeed(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Drop rows from the input dataframe if the DATE column value is not 23:59
    and the HourlyWindSpeed column value is not a float or is NaN.

    Parameters
    ----------
    dataframe : pd.DataFrame
        Input dataframe with 'DATE' and 'HourlyWindSpeed' columns.

    Returns
    -------
    pd.DataFrame
        The dataframe after dropping invalid rows.
    """
    # Filter rows with DATE values not equal to 23:59
    not_2359_rows = (dataframe['DATE'].dt.hour != 23) | (dataframe['DATE'].dt.minute != 59)
    
    # Check if the HourlyWindSpeed is of type float and not NaN
    is_float_and_not_nan = dataframe['HourlyWindSpeed'].apply(lambda x: isinstance(x, float) and not pd.isna(x))
    
    # Combine both conditions using bitwise AND
    invalid_rows = not_2359_rows & ~is_float_and_not_nan

    # Drop the invalid rows
    dataframe = dataframe[~invalid_rows]

    return dataframe

In [ ]:
def generate_missing_dates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate a DataFrame containing missing daily records.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing weather data from January 2009 to June 2015.

    Returns
    -------
    pd.DataFrame
        A DataFrame with missing daily records.
    """
    # Find the minimum and maximum dates in the input DataFrame
    min_date = df['DATE'].min().normalize()
    max_date = df['DATE'].max().normalize()
    # Create a date range from the minimum to maximum date
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    # Generate missing daily records by checking if a record exists for each date
    missing_dates = [
        {'DATE': date.replace(hour=23, minute=59, second=0)}
        for date in date_range
        if not ((df['DATE'] == date.replace(hour=23, minute=59, second=0)).any())
    ]
    return pd.DataFrame(missing_dates)

In [ ]:
def fill_missing_daily_wind_speed(new_df: pd.DataFrame) -> None:
    """
    Fill missing daily wind speed values by calculating the mean hourly wind speed.

    Parameters
    ----------
    new_df : pd.DataFrame
        The input DataFrame containing weather data with missing daily wind speed values.
    """
    # Iterate through the rows of the DataFrame with missing daily wind speed values
    for index, row in new_df.loc[new_df['DATE'].dt.strftime('%H:%M') == '23:59'].iterrows():
        if pd.isna(row['DailyAverageWindSpeed']):
            date = row['DATE'].date()
            # Calculate the mean hourly wind speed for the current date
            hourly_wind_speed_mean = new_df.loc[
                (new_df['DATE'].dt.date == date) & (new_df['DATE'] != row['DATE']) & (~new_df['HourlyWindSpeed'].isna()),
                'HourlyWindSpeed'
            ].mean()
            # Replace the missing daily wind speed value with the calculated mean
            new_df.loc[index, 'DailyAverageWindSpeed'] = hourly_wind_speed_mean

In [ ]:
def get_all_weather_csvs(directory: str) -> List[pd.DataFrame]:
    """
    Load and clean the CSV files, return them as a list of DataFrames.

    Parameters
    ----------
    directory : str
        The directory path containing the CSV files.

    Returns
    -------
    List[pd.DataFrame]
        A list of cleaned DataFrames.
    """
    # List all CSV files in the directory
    csv_files = [entry.name for entry in os.scandir(directory)
        if entry.name.endswith('.csv') and entry.is_file()]
    dfs = []
    # Choose useful columns for the coming analysis
    columns_to_keep = [
        'DATE', 'HourlyPrecipitation', 'HourlyWindSpeed',
        'DailyAverageWindSpeed', 'Sunrise', 'Sunset'
    ]
    
    # Process each CSV file
    for csv_file in csv_files:
        # Join the directory path and CSV file name to form the full file path
        file_path = os.path.join(directory, csv_file)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, usecols=columns_to_keep, parse_dates=['DATE'], engine='python')
        
        # Filter the DataFrame by date range
        df = df[(df['DATE'] >= '2009-01-01 00:00:00') & (df['DATE'] <= '2015-06-30 23:59:59')]
        
        # Drop rows that do not meet the requirement of 'HourlyWindSpeed'.
        df = drop_invalid_rows_for_HourlyWindSpeed(df)
        
        # Generate a DataFrame with missing dates
        missing_df = generate_missing_dates(df)
        
        # Combine the original DataFrame and the missing dates DataFrame
        new_df = pd.concat([df, missing_df], ignore_index=True)
        # Sort the new DataFrame by date
        new_df.sort_values(by='DATE', inplace=True)
        
        # Fill in missing daily wind speed values
        fill_missing_daily_wind_speed(new_df)
        
        # Append the cleaned DataFrame to the list of DataFrames
        dfs.append(new_df)
        
    
    return dfs

In [ ]:
def clean_hourly_precipitation(hourly_data: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the 'HourlyPrecipitation' column of the input DataFrame.

    Parameters
    ----------
    hourly_data : pd.DataFrame
        Input DataFrame containing hourly weather data.

    Returns
    -------
    pd.DataFrame
        DataFrame containing hourly data.
    """
    # Remove 's' from the HourlyPrecipitation values, replace 'T' with 0, and fill missing values with 0
    hourly_data['HourlyPrecipitation'] = hourly_data['HourlyPrecipitation'].str.replace('s', '')
    hourly_data['HourlyPrecipitation'] = hourly_data['HourlyPrecipitation'].replace('T', 0)
    hourly_data['HourlyPrecipitation'].fillna(0, inplace=True)
    # Convert the HourlyPrecipitation column to numeric type
    hourly_data['HourlyPrecipitation'] = pd.to_numeric(hourly_data['HourlyPrecipitation'])
    return hourly_data

In [ ]:
def aggregate_hourly_data(hourly_data: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate the input DataFrame's data to remove multiple reports within an hour.

    Parameters
    ----------
    hourly_data : pd.DataFrame
        Input DataFrame containing hourly weather data.

    Returns
    -------
    pd.DataFrame
        DataFrame with aggregated hourly data.
    """
    # Round the datetime to the nearest hour
    hourly_data['date'] = hourly_data['date'].dt.floor('H')
    # Group by hour and aggregate the data by taking the maximum of hourly_precipitation and hourly_windspeed
    return hourly_data.groupby('date').agg(
        {'hourly_precipitation': 'max', 'hourly_windspeed': 'max'}
    ).reset_index()

In [ ]:
def clean_month_weather_data_hourly(csv_file: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the DataFrame and return only hourly weather data.

    Parameters
    ----------
    csv_file : pd.DataFrame
        Input DataFrame containing weather data.

    Returns
    -------
    pd.DataFrame
        DataFrame with aggregated hourly data.
    """
    # Split the input DataFrame into daily and hourly data
    daily_data = csv_file[(csv_file['DATE'].dt.hour == 23) & (csv_file['DATE'].dt.minute == 59)]
    hourly_data = csv_file.drop(daily_data.index)
    # Select only relevant columns for further processing
    columns_to_keep = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']
    hourly_data = hourly_data[columns_to_keep]

    # Clean the hourly precipitation and wind speed data
    hourly_data = clean_hourly_precipitation(hourly_data)
    # Clean the 'HourlyWindSpeed' column
    hourly_data['HourlyWindSpeed'].fillna(0, inplace=True)
    # Normalize the column names
    hourly_data.columns = ['date', 'hourly_precipitation', 'hourly_windspeed']
    # Aggregate the hourly data to remove multiple reports within an hour
    result = aggregate_hourly_data(hourly_data)

    return result

In [ ]:
def float_to_time_string(value: float) -> str:
    """
    Transform the float value to time string in HH:MM format.

    Parameters
    ----------
    value : float
        Float value representing time.

    Returns
    -------
    str
        Formatted time string in HH:MM format.
    """
    if pd.isna(value):
        return None
    else:
        # Extract hours and minutes from the float value
        hours = int(value // 100)
        minutes = int(value % 100)
    # Return formatted time string
    return f"{hours:02d}:{minutes:02d}"

In [ ]:
def convert_time_to_datetime(row: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert 'Sunrise' and 'Sunset' time strings to datetime objects using the date from the 'DATE' column.

    Parameters
    ----------
    row : Dict[str, Any]
        A dictionary representing a row in the DataFrame containing 'DATE', 'Sunrise', and 'Sunset' columns.

    Returns
    -------
    Dict[str, Any]
        The modified row with 'Sunrise' and 'Sunset' columns converted to datetime objects.
    """
    # Extract the date from the 'DATE' column
    date = row['DATE'].date()
    
    # Check if the 'Sunrise' value is not None
    if row['Sunrise'] is not None:
        # Convert the 'Sunrise' time string to a time object
        sunrise_time = pd.to_datetime(row['Sunrise'], format="%H:%M").time()
        # Combine the date and sunrise_time to create a datetime object
        row['Sunrise'] = pd.to_datetime(f"{date} {sunrise_time}", format="%Y-%m-%d %H:%M:%S")
    
    # Check if the 'Sunset' value is not None
    if row['Sunset'] is not None:
        # Convert the 'Sunset' time string to a time object
        sunset_time = pd.to_datetime(row['Sunset'], format="%H:%M").time()
        # Combine the date and sunset_time to create a datetime object
        row['Sunset'] = pd.to_datetime(f"{date} {sunset_time}", format="%Y-%m-%d %H:%M:%S")

    # Return the modified row
    return row

In [ ]:
def clean_month_weather_data_daily(csv_file: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the DataFrame and return only daily weather data.

    The function processes the input DataFrame and returns a cleaned DataFrame
    with daily weather data. The steps include extracting daily data, selecting
    relevant columns, cleaning sunrise and sunset data, and normalizing column names.

    Parameters
    ----------
    csv_file : pd.DataFrame
        DataFrame containing weather data.

    Returns
    -------
    pd.DataFrame
        DataFrame containing cleaned daily weather data.
    """
    # Filter daily data based on timestamp (hour=23 and minute=59)
    daily_data = csv_file[(csv_file['DATE'].dt.hour == 23) & (csv_file['DATE'].dt.minute == 59)]
    # Define the columns to keep
    columns_to_keep = ['DATE', 'DailyAverageWindSpeed', 'Sunrise', 'Sunset']
    daily_data = daily_data[columns_to_keep]
    
    # Clean the  "Sunrise" and "Sunset" columns
    # Convert 'Sunrise' and 'Sunset' columns to strings in HH:MM format
    daily_data['Sunrise'] = daily_data['Sunrise'].apply(float_to_time_string)
    daily_data['Sunset'] = daily_data['Sunset'].apply(float_to_time_string)
    # Convert 'Sunrise' and 'Sunset' columns to datetime
    daily_data = daily_data.apply(convert_time_to_datetime, axis=1)
    
    # Normalize the column names
    daily_data.columns = ['date', 'daily_average_windspeed', 'sunrise', 'sunset']
    # Round the datetime column in a DataFrame to the nearest day.
    daily_data["date"] = pd.to_datetime(daily_data["date"]).dt.date

    return daily_data

In [ ]:
def load_and_clean_weather_data() -> tuple:
    """
    Load and clean weather data, return hourly and daily records as DataFrames.

    The function loads weather data from CSV files, cleans the data, and returns
    two DataFrames containing hourly and daily weather data respectively.

    Returns
    -------
    tuple
        Tuple containing two DataFrames: (hourly_data, daily_data).
    """
    # Load weather CSV files
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)

    hourly_dataframes = []
    daily_dataframes = []

    # Process each CSV file
    for csv_file in weather_csv_files:
        # Clean hourly and daily weather data
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        # Append DataFrames to respective lists
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)

    # Create two DataFrames with hourly and daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)

    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
daily_weather_data.head()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# Create the table schema
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    hourly_precipitation FLOAT,
    hourly_windspeed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    daily_average_windspeed FLOAT,
    sunrise DATETIME,
    sunset DATETIME
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    tip_amount FLOAT,
    distance FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict: Dict[str, pd.DataFrame]) -> None:
    """
    Write DataFrames to their corresponding database tables.

    Parameters
    ----------
    table_to_df_dict : Dict[str, pd.DataFrame]
        A dictionary where keys represent table names and values represent DataFrames to be written to those tables.
    """
    # Iterate through the table name and DataFrame pairs in the dictionary
    for table, dataframe in table_to_df_dict.items():
        # Write the DataFrame to the corresponding table in the database
        dataframe.to_sql(table, con=engine, if_exists='append', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query: str, outfile: str) -> None:
    """Helper function to write the queries to file.
    
    Parameters
    ----------
    query : str
        The query statement.
    outfile : str
        The relative path of the output file.
    
    """
    with open(outfile, "w") as f:
        f.write(query)

### Query 1

In [ ]:
QUERY_1_FILENAME = f"{QUERY_DIRECTORY}/taxi-popularity-by-hour.sql"

QUERY_1 = """
SELECT 
    strftime('%H', pickup_datetime), 
    COUNT(*) AS Count
FROM
    taxi_trips
GROUP BY 
    strftime('%H', pickup_datetime)
ORDER BY 
    Count DESC;
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2

In [ ]:
QUERY_2_FILENAME = "uber_trips_weekday_popularity.sql"

QUERY_2 = """
SELECT
    CASE
        WHEN strftime('%w', pickup_datetime) = '0' THEN 'Sunday'
        WHEN strftime('%w', pickup_datetime) = '1' THEN 'Monday'
        WHEN strftime('%w', pickup_datetime) = '2' THEN 'Tuesday'
        WHEN strftime('%w', pickup_datetime) = '3' THEN 'Wednesday'
        WHEN strftime('%w', pickup_datetime) = '4' THEN 'Thursday'
        WHEN strftime('%w', pickup_datetime) = '5' THEN 'Friday'
        WHEN strftime('%w', pickup_datetime) = '6' THEN 'Saturday'
    END AS weekday,
    COUNT(*) AS popularity
FROM
    uber_trips
GROUP BY
    weekday
ORDER BY
    popularity DESC;
"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3

In [ ]:
QUERY_3_FILENAME = f"{QUERY_DIRECTORY}/trip_distance_95_percentile_july_2013.sql"

QUERY_3 = """
WITH all_trips AS (SELECT pickup_datetime, distance FROM taxi_trips
                   UNION ALL
                   SELECT pickup_datetime, distance FROM uber_trips)
SELECT
    distance
FROM
    all_trips
WHERE
    strftime('%Y', pickup_datetime) = '2013' AND strftime('%m', pickup_datetime) = '07'
ORDER BY
    distance
lIMIT 1
OFFSET (SELECT COUNT(*)
        FROM all_trips
        WHERE strftime('%Y', pickup_datetime) = '2013' AND strftime('%m', pickup_datetime) = '07') * 95 / 100 - 1;
"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4

In [ ]:
QUERY_4_FILENAME = "top_10_busiest_days_with_avg_distance_2009.sql"

QUERY_4 = """
SELECT
    date(pickup_datetime) AS pickup_date,
    COUNT(*) AS total_rides,
    AVG(distance) AS avg_distance
FROM
    (
        SELECT pickup_datetime, distance
        FROM uber_trips
        WHERE strftime('%Y', pickup_datetime) = '2009'
        UNION ALL
        SELECT pickup_datetime, distance
        FROM taxi_trips
        WHERE strftime('%Y', pickup_datetime) = '2009'
    )
GROUP BY
    pickup_date
ORDER BY
    total_rides DESC, pickup_date
LIMIT 10;
"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5

In [ ]:
QUERY_5_FILENAME = f"{QUERY_DIRECTORY}/windiest_10_days_with_num_of_trips.sql"

QUERY_5 = """
WITH all_trips AS (SELECT pickup_datetime, distance FROM taxi_trips
                   UNION ALL
                   SELECT pickup_datetime, distance FROM uber_trips)
SELECT
    w.date, w.daily_average_windspeed, COUNT(*) as num_trips
FROM
    daily_weather AS w
    JOIN
    all_trips AS t ON strftime('%Y-%m-%d', w.date) = strftime('%Y-%m-%d', t.pickup_datetime)
WHERE
    w.date BETWEEN '2014-01-01' AND '2014-12-31'
GROUP BY
    w.date
ORDER BY
    w.daily_average_windspeed DESC
LIMIT 10;
"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6

In [ ]:
QUERY_6_FILENAME = "hurricane_sandy_hourly_trips_and_weather.sql"

QUERY_6 = """
WITH RECURSIVE
    date_range (hour, n) AS (
        SELECT datetime(strftime('%Y-%m-%d %H', '2012-10-22 00:00:00', '+0 hours') || ':00:00') AS hour, 0 AS n
        UNION ALL
        SELECT datetime(strftime('%Y-%m-%d %H', '2012-10-22 00:00:00', '+' || (n + 1) || ' hours') || ':00:00') AS hour, n + 1
        FROM date_range
        WHERE n < 383
    ),
    trips AS (
        SELECT 
            datetime(strftime('%Y-%m-%d %H', pickup_datetime) || ':00:00') AS pickup_hour,
            COUNT(*) AS trip_count
        FROM (
            SELECT pickup_datetime FROM uber_trips
            UNION ALL
            SELECT pickup_datetime FROM taxi_trips
        )
        WHERE pickup_datetime BETWEEN '2012-10-22 00:00:00' AND '2012-11-06 23:59:59'
        GROUP BY pickup_hour
    )
SELECT
    dr.hour AS date_hour_range,
    COALESCE(t.trip_count, 0) AS trip_count,
    COALESCE(hw.hourly_precipitation, 0.0) AS hourly_precipitation,
    COALESCE(hw.hourly_windspeed, 0.0) AS hourly_windspeed
FROM
    date_range dr
LEFT JOIN
    trips t ON dr.hour = t.pickup_hour
LEFT JOIN
    hourly_weather hw ON dr.hour = datetime(strftime('%Y-%m-%d %H', hw.date) || ':00:00')
ORDER BY
    dr.hour;
"""

In [ ]:
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

## Part 4: Visualizing the Data

In [ ]:
def remove_outliers_std(df: pd.DataFrame, column: str, threshold: float = 3) -> pd.DataFrame:
    """
    Remove the outliers from the input dataframe based on the standard deviation.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    column : str
        The column name to remove outliers from.
    threshold : float, optional
        The number of standard deviations to use as the threshold. Defaults to 3.

    Returns
    -------
    pd.DataFrame
        The dataframe without outliers.
    """
    mean = df[column].mean()
    std = df[column].std()

    lower_bound = mean - threshold * std
    upper_bound = mean + threshold * std

    df_no_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return df_no_outliers

### Visualization 1

In [ ]:
def plot_num_taxi_by_hour(dataframe: pd.DataFrame) -> None:
    """Plot the popularity of Yellow Taxi rides by each hour.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The dataframe containing data with the hour and number of taxi rides.
    
    """
    figure, axes = plt.subplots(figsize=(18, 8))
    
    values = dataframe['Count']
    
    axes.bar(dataframe['Hour'], values)
    axes.set_title("Popularity of Yellow Taxi rides for each hour of the day")
    axes.set_xlabel("Hour")
    axes.set_ylabel("Number of Yellow Taxi Rides")
    
    plt.show()

In [ ]:
def data_num_taxi_by_hour() -> pd.DataFrame:
    """Obtain the popularity of Yellow Taxi rides by each hour.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe including hours and corresponding number of yellow taxi rides.
    
    """
    query = """
        SELECT 
            strftime('%H', pickup_datetime) AS Hour, 
            COUNT(*) AS Count
        FROM
            taxi_trips
        GROUP BY 
            strftime('%H', pickup_datetime);
    """
    dataframe = pd.read_sql_query(query, engine)
    return dataframe

In [ ]:
df_num_taxi_by_hour = data_num_taxi_by_hour()
plot_num_taxi_by_hour(df_num_taxi_by_hour)

### Visualization 2

In [ ]:
def calculate_monthly_statistics(dataframe: pd.DataFrame) -> Tuple[pd.Series, pd.Series, pd.Series]:
    """Calculate monthly statistics, including mean, standard deviation, and confidence interval.
    
    Parameters
    ----------
    dataframe : pd.DataFrame
        DataFrame containing distance data with a 'month' column.

    Returns
    -------
    tuple
        Tuple containing mean distances, standard deviations, and confidence intervals (means, std_devs, ci)
    """
    df = dataframe.sort_values(by='month')
    means = df.groupby('month')['distance'].mean()
    std_devs = df.groupby('month')['distance'].std()
    ci = 1.645 * (std_devs / np.sqrt(df.groupby('month')['distance'].count()))
    
    return means, std_devs, ci

In [ ]:
def set_plot_labels(ax: Axes, means: pd.Series, ci: pd.Series) -> None:
    """Set plot labels for the bar plot, including x-axis, y-axis, and title.
    
    Parameters
    ----------
    ax : Axes
        Axes object of the bar plot.
    means : pd.Series
        Mean distances for each month.
    ci : pd.Series
        Confidence intervals for each month.
    """
    ax.set_xlabel('Month')
    ax.set_ylabel('Average Distance (km)')
    ax.set_title('The average distance of trips for each month\n(2009.1-2015.6) with 90% confidence intervals')
    ax.set_ylim(means.min() - ci.max() - 0.05, means.max() + ci.max() * 1.5)
    ax.yaxis.set_ticks(np.arange(means.min() - ci.max(), means.max() + ci.max() * 1.5, 0.1))
    
    # Set custom x-axis labels
    month_labels = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    ax.set_xticks(range(len(month_labels)))
    ax.set_xticklabels(month_labels, rotation=0)  # Set rotation to 0 for horizontal labels


In [ ]:
def add_bar_labels(ax: Axes, bars: BarContainer, means: pd.Series, ci: pd.Series) -> None:
    """Add labels to the bars in the bar plot, including bar height and confidence interval limits.
    
    Parameters
    ----------
    ax : Axes
        Axes object of the bar plot.
    bars : BarContainer
        Bars of the bar plot.
    means : pd.Series
        Mean distances for each month.
    ci : pd.Series
        Confidence intervals for each month.
    """
    for i, bar in enumerate(bars.patches):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height - 0.02, f"{height:.2f}", ha='center', va='top', fontsize=8, color='white')

        upper_limit = means.iloc[i] + ci.iloc[i]
        lower_limit = means.iloc[i] - ci.iloc[i]
        ax.text(bar.get_x() + bar.get_width() / 2, upper_limit + 0.005, f"{upper_limit:.2f}", ha='center', va='bottom', fontsize=8, color='black')
        ax.text(bar.get_x() + bar.get_width() / 2, lower_limit - 0.005, f"{lower_limit:.2f}", ha='center', va='top', fontsize=8, color='black')
    

In [ ]:
def plot_avg_trip_distance_per_month(dataframe: pd.DataFrame) -> None:
    """Plots a bar chart showing the average trip distance per month, based on the data in the given DataFrame.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The DataFrame containing trip distance data. It should have at least two columns: 'month' and 'distance'.
    """
   # Calculate means, std_devs, and ci
    means, std_devs, ci = calculate_monthly_statistics(dataframe)

    # Create a bar plot with error bars representing confidence intervals
    fig, ax = plt.subplots(figsize=(15, 8))
    bars = means.plot(kind='bar', yerr=ci, capsize=4, ax=ax)

    set_plot_labels(ax, means, ci)
    add_bar_labels(ax, bars, means, ci)

    plt.show()

In [ ]:
def get_data_for_visual_2() -> pd.DataFrame:
    """Retrieve the data for plotting average trip distance per month.
    """
    QUERY = """
    SELECT strftime('%m', pickup_datetime) as month, distance as distance
    FROM (
      SELECT pickup_datetime, distance FROM taxi_trips
      UNION ALL
      SELECT pickup_datetime, distance FROM uber_trips
    ) as combined_trips;
    """
    result = engine.execute(QUERY).fetchall()
    df = pd.DataFrame(result, columns=['month', 'distance'])
    return df

In [ ]:
some_dataframe = get_data_for_visual_2()
plot_avg_trip_distance_per_month(some_dataframe)

### Visualization 3

In [ ]:
def plot_airport_dropoff_by_weekday(dataframe_1: pd.DataFrame, dataframe_2: pd.DataFrame,
                                    dataframe_3: pd.DataFrame):
    """Plot the popularity of rides dropping off at airports by weekdays.
    
    Parameters
    ----------
    dataframe1 : pandas.DataFrame
        The dataframe containing data with dropoff coordinate within a airport.
    dataframe2 : pandas.DataFrame
        The dataframe containing data with dropoff coordinate within a airport.
    dataframe3 : pandas.DataFrame
        The dataframe containing data with dropoff coordinate within a airport.
        
    """
    figure, axes = plt.subplots(3, 1, figsize=(20, 10))
    
    # Plot the bars for each data frame on separate subplots
    axes[0].bar(dataframe_1['weekday'], dataframe_1['Count'])
    axes[0].set_title('LGA Airport')
    axes[0].set_xlabel("Weekday")
    axes[0].set_ylabel('Number of Rides')
    
    axes[1].bar(dataframe_2['weekday'], dataframe_2['Count'])
    axes[1].set_title('JFK Airport')
    axes[1].set_xlabel("Weekday")
    axes[1].set_ylabel('Number of Rides')
    
    axes[2].bar(dataframe_3['weekday'], dataframe_3['Count'])
    axes[2].set_title('EWR Airport')
    axes[2].set_xlabel("Weekday")
    axes[2].set_ylabel('Number of Rides')
    
    figure.suptitle('Popularity of rides for each day of the week', fontsize=16)
    
    # Adjust the subplots spacing and padding
    figure.subplots_adjust(hspace=0.4, top=0.9, bottom=0.1)
    
    plt.show()

In [ ]:
def get_airport_dropoff_by_weekday(coords: tuple) -> pd.DataFrame:
    """Obtain the popularity of rides dropping off at a airport by weekdays.
    
    Parameters
    ----------
    coords : pandas.DataFrame
        The coordinates of the airport.
    
    Returns
    -------
    pd.DataFrame
        A dataframe including weekdays and corresponding rides that dropped off at the given airport.
        
    """
    query = f"""
        WITH all_trips AS (SELECT pickup_datetime, dropoff_longitude, dropoff_latitude FROM taxi_trips
                           UNION ALL
                           SELECT pickup_datetime, dropoff_longitude, dropoff_latitude FROM uber_trips)
        SELECT
            CASE
                WHEN strftime('%w', pickup_datetime) = '0' THEN 'Sunday'
                WHEN strftime('%w', pickup_datetime) = '1' THEN 'Monday'
                WHEN strftime('%w', pickup_datetime) = '2' THEN 'Tuesday'
                WHEN strftime('%w', pickup_datetime) = '3' THEN 'Wednesday'
                WHEN strftime('%w', pickup_datetime) = '4' THEN 'Thursday'
                WHEN strftime('%w', pickup_datetime) = '5' THEN 'Friday'
                WHEN strftime('%w', pickup_datetime) = '6' THEN 'Saturday'
            END AS weekday,
            COUNT(*) AS Count
        FROM
            all_trips
        WHERE
            dropoff_longitude BETWEEN {coords[0][1]} AND {coords[1][1]}
            AND
            dropoff_latitude BETWEEN {coords[0][0]} AND {coords[1][0]}
        GROUP BY
            weekday
    """
    dataframe = pd.read_sql_query(query, engine)
    
    # Define the order of weekdays as a list
    order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    # Convert the "weekday" column to a categorical data type with the custom order
    dataframe['weekday'] = pd.Categorical(dataframe['weekday'], categories=order, ordered=True)
    # Sort the dataframe by the "weekday" column
    dataframe = dataframe.sort_values('weekday')
    
    return dataframe

In [ ]:
df_2_LGA = get_airport_dropoff_by_weekday(LGA_BOX_COORDS)
df_2_JFK = get_airport_dropoff_by_weekday(JFK_BOX_COORDS)
df_2_EWR = get_airport_dropoff_by_weekday(EWR_BOX_COORDS)
plot_airport_dropoff_by_weekday(df_2_LGA, df_2_JFK, df_2_EWR)

### Visualization 4

In [ ]:
def extract_coordinates(dataframe: pd.DataFrame) -> List[Tuple[float, float]]:
    """Extract pickup and dropoff coordinates from the input DataFrame.

    Parameters
    ----------
    dataframe : pd.DataFrame
        DataFrame containing pickup and dropoff latitude and longitude data.

    Returns
    -------
    list
        A list of pickup and dropoff coordinates as tuples (latitude, longitude).
    """
    pickup_coords = dataframe[['pickup_latitude', 'pickup_longitude']].values.tolist()
    dropoff_coords = dataframe[['dropoff_latitude', 'dropoff_longitude']].values.tolist()
    return pickup_coords + dropoff_coords

In [ ]:
def heatmap_pickup_dropoff_locations(dataframe: pd.DataFrame) -> folium.Map:
    """Create a heatmap of pickup and dropoff locations based on the input DataFrame.

    Parameters
    ----------
    dataframe : pd.DataFrame
        DataFrame containing pickup and dropoff latitude and longitude data.

    Returns
    -------
    folium.Map
        A folium map object containing the heatmap.
    """
    # Create a base map using the mean latitude and longitude values from the input DataFrame.
    m = folium.Map(location=[(dataframe['pickup_latitude'].mean() + dataframe['dropoff_latitude'].mean()) / 2,
                              (dataframe['pickup_longitude'].mean() + dataframe['pickup_longitude'].mean()) / 2],
                   zoom_start=10)
    
    # Get all coordinates of pickup and dropoff points
    all_coords = extract_coordinates(dataframe)
    
    # Set heatmap parameters
    heatmap_params = {
        'radius': 12,
        'max_zoom': 18,
        'gradient': {0.2: 'blue', 0.4: 'green', 0.6: 'yellow', 0.8: 'orange', 1.0: 'red'}
    }
    
    # Create and add the heatmap to the map
    heatmap = HeatMap(all_coords, **heatmap_params)
    heatmap.add_to(m)
    
    return m

In [ ]:
def get_data_for_visual_4() -> pd.DataFrame:
    """Retrieve the data for plotting a heatmap of pickup and dropoff locations.

    Returns
    -------
    pd.DataFrame
        DataFrame containing pickup and dropoff latitude and longitude data.
    """
    QUERY = """
    SELECT
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude
    FROM
        taxi_trips
    UNION ALL
    SELECT
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude
    FROM
        uber_trips;
    """
    result = engine.execute(QUERY).fetchall()
    df = pd.DataFrame(result, columns=['pickup_longitude', 'pickup_latitude', "dropoff_longitude", "dropoff_latitude"])
    return df

In [ ]:
some_dataframe = get_data_for_visual_4()
heatmap_pickup_dropoff_locations(some_dataframe)

### Visualization 5

In [ ]:
def plot_tip_distance_taxi(dataframe: pd.DataFrame) -> None:
    """Plot the scatter plot of tip amount versus the distance.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The dataframe containing data with the tip amount and the distance of rides.
    
    """
    figure, axes = plt.subplots(figsize=(20, 10))
    
    # create a scatter plot
    axes.scatter(x=dataframe['distance'], y=dataframe['tip_amount'])
    
    # set the x and y axis labels
    axes.set_xlabel('Distance')
    axes.set_ylabel('Tip Amount')
    
    # set the plot title
    axes.set_title("Tip Amount vs. Distance")
    
    plt.show()

In [ ]:
def plot_tip_distance_taxi_mean(dataframe: pd.DataFrame) -> None:
    """Plot the scatter plot of the means of tip amount versus the distance.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The dataframe containing data with the tip amount and the distance of rides.
    
    """
    figure, axes = plt.subplots(figsize=(20, 10))
    
    # divide the distance into 100 bins and calculate the mean of tip_amount for each bin
    mean_tip_amount = dataframe.groupby(pd.cut(dataframe['distance'], bins=100))['tip_amount'].mean()
    bin_edges = pd.cut(dataframe['distance'], bins=100, retbins=True)[1]
    distance = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # create a line plot of mean tip_amount vs distance
    axes.scatter(distance, mean_tip_amount)
    
    # set the x and y axis labels
    axes.set_xlabel('Distance')
    axes.set_ylabel('Mean Tip Amount')
    
    # set the plot title
    axes.set_title("Mean Tip Amount vs. Distance")
    
    plt.show()

In [ ]:
def plot_tip_distance_taxi_combined(dataframe: pd.DataFrame) -> None:
    """Plot the scatter plot of tip amount versus the distance and the mean tip amount versus the distance.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The dataframe containing data with the tip amount and the distance of rides.
    
    """
    figure, axes = plt.subplots(figsize=(20, 10))
    
    # create a scatter plot of original data
    axes.scatter(x=dataframe['distance'], y=dataframe['tip_amount'], label='Original Data', alpha=0.5, color='dodgerblue')
    
    # divide the distance into 100 bins and calculate the mean of tip_amount for each bin
    mean_tip_amount = dataframe.groupby(pd.cut(dataframe['distance'], bins=100))['tip_amount'].mean()
    bin_edges = pd.cut(dataframe['distance'], bins=100, retbins=True)[1]
    distance = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # create a secondary y-axis for the mean tip_amount
    ax2 = axes.twinx()
    
    # create a line plot of mean tip_amount vs distance on the secondary y-axis
    ax2.scatter(distance, mean_tip_amount, color='darkorange', label='Mean Tip Amount', marker='o')
    
    # set the x and y axis labels
    axes.set_xlabel('Distance')
    axes.set_ylabel('Tip Amount')
    ax2.set_ylabel('Mean Tip Amount')
    
    # set the plot title
    axes.set_title("Tip Amount vs. Distance and Mean Tip Amount vs. Distance")
    
    # add legends for both axes
    axes.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.show()

In [ ]:
def data_tip_distance_taxi() -> pd.DataFrame:
    """Obtain tip amounts and distances of yellow taxi rides.
    
    Returns
    -------
    pandas.DataFrame
        A dataframe including tip amounts and distances of yellow taxi rides.
    
    """
    query = """
        SELECT 
            tip_amount,
            distance
        FROM
            taxi_trips
    """
    dataframe = pd.read_sql_query(query, engine)
    return dataframe

In [ ]:
df_tip_distance = data_tip_distance_taxi()
df_tip_distance = remove_outliers_std(df_tip_distance, 'tip_amount')
df_tip_distance = remove_outliers_std(df_tip_distance, 'distance')
plot_tip_distance_taxi_combined(df_tip_distance)

### Visualization 6

In [ ]:
def calculate_binned_means(dataframe: pd.DataFrame, x_col: str, y_col: str, num_bins: int) -> pd.DataFrame:
    """
    Calculate the mean values for specified number of bins.
    
    Parameters
    ----------
    dataframe : pd.DataFrame
        Input dataframe.
    x_col : str
        The x-axis column name.
    y_col : str
        The y-axis column name.
    num_bins : int
        The number of bins to be created.
    
    Returns
    -------
    pd.DataFrame
        A dataframe containing the binned mean values.
    """
    # Calculate the bins and bin the data
    bins = np.linspace(dataframe[x_col].min(), dataframe[x_col].max(), num_bins + 1)
    dataframe['binned'] = pd.cut(dataframe[x_col], bins)
    
    # Calculate the mean value for each bin and clean up the output dataframe
    binned_means = dataframe.groupby('binned')[y_col].mean().reset_index()
    binned_means[x_col] = [(interval.left + interval.right) / 2 for interval in binned_means['binned']]
    del binned_means['binned']
    
    return binned_means

In [ ]:
def plot_original_data(ax: matplotlib.axes.Axes, dataframe: pd.DataFrame, x_col: str, y_col: str) -> None:
    """
    Plot the original data as a scatter plot.
    
    Parameters
    ----------
    ax : matplotlib.axes.Axes
        The axes object to plot on.
    dataframe : pd.DataFrame
        Input dataframe.
    x_col : str
        The x-axis column name.
    y_col : str
        The y-axis column name.
    """
    # Plot the original data
    ax.scatter(x=dataframe[x_col], y=dataframe[y_col], alpha=0.5, color='#89CFF0', label='Original Data')
    ax.set_xlabel(f"{x_col} (inches to hundredths)")
    ax.set_ylabel(f"{y_col} (dollars)")
    ax.yaxis.label.set_color('#89CFF0')
    ax.tick_params(axis='y', colors='#89CFF0')
    ax.legend(loc='upper left')

In [ ]:
def plot_binned_means(ax: matplotlib.axes.Axes, binned_means: pd.DataFrame, x_col: str, y_col: str) -> None:
    """
    Plot the binned mean values as a scatter plot.
    
    Parameters
    ----------
    ax : matplotlib.axes.Axes
        The axes object to plot on.
    binned_means : pd.DataFrame
        The dataframe containing binned mean values.
    x_col : str
        The x-axis column name.
    y_col : str
        The y-axis column name.
    """
    # Plot the binned means
    ax.scatter(x=binned_means[x_col], y=binned_means[y_col], color='#FFA07A', label='Binned Means')
    ax.set_ylabel('Binned Means(dollars)')
    ax.yaxis.label.set_color('#FFA07A')
    ax.tick_params(axis='y', colors='#FFA07A')
    ax.legend(loc='upper right')

In [ ]:
# To Understand the relationship between hourly precipitation and tip amount, we should not only use the raw data but also use the binned data
# The raw data are not enough for us to get a reasonable conclusion
def plot_scatter_with_original_data_and_binned_means(dataframe: pd.DataFrame, x_col: str = "hourly_precipitation", 
                                                     y_col: str = "tip", num_bins: int = 100) -> None:
    """
    Plot the original data and binned mean values on the same plot.
    
    Parameters
    ----------
    dataframe : pd.DataFrame
        Input dataframe.
    x_col : str, optional
        The x-axis column name. Defaults to "hourly_precipitation".
    y_col : str, optional
        The y-axis column name. Defaults to "tip".
    num_bins : int, optional
        The number of bins to be created. Defaults to 100.
    """
    binned_means = calculate_binned_means(dataframe, x_col, y_col, num_bins)
    fig, ax1 = plt.subplots()
    plot_original_data(ax1, dataframe, x_col, y_col)
    ax2 = ax1.twinx()
    plot_binned_means(ax2, binned_means, x_col, y_col)
    plt.show()

In [ ]:
def get_data_for_visual_6() -> pd.DataFrame:
    """Retrieve and preprocess data for Visual 6.
    
    This function queries hourly weather and taxi trips data, merges the two dataframes on the 'time' column, 
    and removes outliers from the 'hourly_precipitation' and 'tip' columns using the standard deviation method.

    Returns
    -------
    pd.DataFrame
        A preprocessed dataframe with columns 'hourly_precipitation' and 'tip' without outliers.
    """
    QUERY1 =""" 
    SELECT strftime('%Y-%m-%d %H'||":00:00", h.date) AS time, h.hourly_precipitation AS hourly_precipitation
    FROM hourly_weather h
    """
    QUERY2 ="""
    SELECT strftime('%Y-%m-%d %H'||":00:00", t.pickup_datetime) AS time, t.tip_amount AS tip
    FROM taxi_trips t
    """
    hourly_weather_data = pd.read_sql_query(QUERY1, engine)
    taxi_trips_data = pd.read_sql_query(QUERY2, engine)
    
    df = pd.merge(hourly_weather_data,taxi_trips_data,on="time" )
    df = df.drop("time",axis=1)
    # Delete the outliers
    df_no_outliers = remove_outliers_std(df, 'hourly_precipitation')
    df_no_outliers = remove_outliers_std(df_no_outliers, 'tip')
    return df

In [ ]:
some_dataframe = get_data_for_visual_6()
plot_scatter_with_original_data_and_binned_means(some_dataframe)
# We can observe that the binned means of tips remain relatively stable when the hourly precipitation is below 0.2. 
# As the hourly precipitation increases, the binned means of tips fluctuate more dramatically. 
# When the hourly precipitation exceeds 1.2, the binned means of tips approach zero.

# Extra Credits

### Visualization: The relationship between daytime duration and average travel distance

In [ ]:
# Create the schema for the "sunrise and sunset table"
SUNRISE_AND_SUNSET_SCHEMA = """
CREATE TABLE IF NOT EXISTS sunrise_and_sunset (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    sunrise DATETIME,
    sunset DATETIME
)
"""

In [ ]:
with engine.connect() as connection:
    connection.execute(SUNRISE_AND_SUNSET_SCHEMA)

In [ ]:
# Get the sunrise_and_sunset_data from daily_weather_data
sunrise_and_sunset_data = daily_weather_data[["date", "sunrise", "sunset"]]
# Drop the useless rows
sunrise_and_sunset_data = sunrise_and_sunset_data.dropna()

In [ ]:
# Create a dictionry for writting the data into the table
map_table_name_to_dataframe_for_extra_credits_question = {
    "sunrise_and_sunset": sunrise_and_sunset_data
}

In [ ]:
# Write table into the new table
write_dataframes_to_table(map_table_name_to_dataframe_for_extra_credits_question)

In [ ]:
def get_the_data_for_this_visualization_problem() -> pd.DataFrame:
    """Fetch data for the visualization problem.
    
    Returns
    -------
    pd.DataFrame
        A dataframe containing date, daytime duration, and average distance.
    """
    Query = """
    WITH daily_weather_duration AS (
        SELECT date, 
               CAST((julianday(sunset) - julianday(sunrise)) * 24 * 60 AS REAL) AS daytime_duration
        FROM sunrise_and_sunset
    ),

    combined_trips AS (
        SELECT pickup_datetime, distance, date(pickup_datetime) AS date
        FROM taxi_trips
        WHERE pickup_datetime IS NOT NULL AND distance IS NOT NULL
        UNION ALL
        SELECT pickup_datetime, distance, date(pickup_datetime) AS date
        FROM uber_trips
        WHERE pickup_datetime IS NOT NULL AND distance IS NOT NULL
    ),

    trips_with_daytime_duration AS (
        SELECT c.pickup_datetime, c.distance, c.date, d.daytime_duration
        FROM combined_trips c
        JOIN daily_weather_duration d ON c.date = d.date
    )

    SELECT date, 
           AVG(daytime_duration) AS daytime_duration_minutes,
           AVG(distance) AS avg_distance
    FROM trips_with_daytime_duration
    GROUP BY date
    ORDER BY date;
    """
    df = pd.read_sql_query(Query, engine)
    return df

In [ ]:
def calculate_binned_means(df: pd.DataFrame, x_col: str, y_col: str, num_bins: int) -> pd.DataFrame:
    """Calculate binned means for the specified columns.
    
    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe.
    x_col : str
        The name of the x-axis column.
    y_col : str
        The name of the y-axis column.
    num_bins : int
        The number of bins to use.
    
    Returns
    -------
    pd.DataFrame
        A dataframe containing the binned means.
    """
    bin_edges = pd.cut(df[x_col], bins=num_bins, retbins=True)[1]
    binned_means = df.groupby(pd.cut(df[x_col], bins=bin_edges)).mean()
    binned_means = binned_means[[x_col, y_col]].reset_index(drop=True)
    return binned_means

In [ ]:
def plot_daytime_duration_vs_avg_distance(dataframe: pd.DataFrame, x_col: str = "daytime_duration_minutes", 
                                          y_col: str = "avg_distance", num_bins: int = 20) -> None:
    """Plot daytime duration vs average distance.
    
    Parameters
    ----------
    dataframe : pd.DataFrame
        The input dataframe.
    x_col : str, optional
        The name of the x-axis column, by default "daytime_duration_minutes".
    y_col : str, optional
        The name of the y-axis column, by default "avg_distance".
    num_bins : int, optional
        The number of bins to use, by default 20.
    """
    df_no_outliers = remove_outliers_std(dataframe, x_col)
    df_no_outliers = remove_outliers_std(df_no_outliers, y_col)
    
    binned_means = calculate_binned_means(df_no_outliers, x_col, y_col, num_bins)
    
    fig, ax1 = plt.subplots()
    
    # Plot original data
    ax1.scatter(x=df_no_outliers[x_col], y=df_no_outliers[y_col], alpha=0.5, color='#89CFF0', label='Original Data')
    ax1.set_xlabel(f"{x_col} (minutes)")
    ax1.set_ylabel(f"{y_col} (miles)")
    ax1.yaxis.label.set_color('#89CFF0')
    ax1.tick_params(axis='y', colors='#89CFF0')
    ax1.legend(loc='upper left')

    # Plot binned means
    ax2 = ax1.twinx()
    ax2.scatter(x=binned_means[x_col], y=binned_means[y_col], color='#FFA07A', label='Binned Means')
    ax2.set_ylabel('Binned Means (miles)')
    ax2.yaxis.label.set_color('#FFA07A')
    ax2.tick_params(axis='y', colors='#FFA07A')
    ax2.legend(loc='upper right')
    
    plt.title("Daytime Duration vs Average Distance")
    plt.show()

In [ ]:
dataframe = get_the_data_for_this_visualization_problem()
plot_daytime_duration_vs_avg_distance(dataframe)
# We can see that when daytime duration increases, the average distance of trips goes up too.